# Use TI SDK For Tensorboard

本例通过Mnist例子介绍TI SDK如何使用Tensorboard查看Tensorflow的训练模型

### 下载训练数据

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

train_data_path = 'train-data'
data_sets = input_data.read_data_sets('train-data', dtype=tf.uint8, reshape=False, validation_size=5000)

### TFRecord数据转换

In [ ]:
"""Converts MNIST data to TFRecords file format with Example protos."""
import os
import tensorflow as tf


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def convert_to(data_set, name
               , directory):
    """Converts a dataset to tfrecords."""
    images = data_set.images
    labels = data_set.labels
    num_examples = data_set.num_examples

    if images.shape[0] != num_examples:
        raise ValueError('Images size %d does not match label size %d.' %
                         (images.shape[0], num_examples))
    rows = images.shape[1]
    cols = images.shape[2]
    depth = images.shape[3]

    filename = os.path.join(directory, name + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(int(labels[index])),
            'image_raw': _bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())
    writer.close()


convert_to(data_sets.train, 'train', train_data_path)
convert_to(data_sets.validation, 'validation', train_data_path)
convert_to(data_sets.test, 'test', train_data_path)

### cos数据上传

In [ ]:
from ti import session
ti_session = session.Session()
inputs = ti_session.upload_data(path=train_data_path, key_prefix="train-data/mnist")

### Tensorboard模式

1. 您可在训练任务fit函数中传入参数run_tensorboard_locally，该值为True时，TI SDK将会在提交环境中启动Tensorboard服务；
2. 您的训练代码需要处理Tensorflow检查点逻辑，写入到/opt/ml/checkpoints/目录中；TI SDK后台将会自动同步/opt/ml/checkpoints/至cos://{training_bucket}/{training_job_name}/output/checkpoints；其中{training_bucket}和{training_job_name}分别为训练任务的cos桶名称和任务名称；
3. Tensorboard服务默认监听端口为6006，若6006端口不可用时，将会以此从端口6007至端口6105范围尝试可用端口，您可在训练日志输出查看Tensorboard服务端口port；
4. 若提交环境为Notebook环境时，您可通过浏览器中输入域名 https://{notebook_url}/proxy/port/ ，其中{notebook_url}为您的notebook域名，即Notebook的域名后加入后缀/proxy/port/访问Tensorboard服务；
5. 若提交环境为本地环境时，您可通过浏览器中输入本机域名 http://localhost:port ，访问Tensorboard服务；

### 训练任务提交

In [ ]:
from __future__ import absolute_import, print_function

import sys
from ti import session
from ti.tensorflow import TensorFlow

# 初始化
ti_session = session.Session()
hyperparameters = {'train-steps': 5000, "evaluation-steps":1000}

# 创建一个Tensorflow Estimator
tf_estimator = TensorFlow(role="TIONE_QCSRole",
                          train_instance_count=1,
                          train_instance_type='TI.SMALL2.1core2g',
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparameters,
                          framework_version='1.14.0',
                          entry_point='mnist.py',
                          source_dir='train-code')

# 提交Tensorflow训练任务，并使用tensorboard模式
tf_estimator.fit(inputs, run_tensorboard_locally=True)